In [1]:
from pathlib import Path as pth
from pathlib import PurePath as ppth

from random import random, uniform, randint

from src import config


project_path = pth.cwd()

blender_path = "C:/Program Files/Blender Foundation/Blender 3.4/blender" # path to blender exe (without .exe)
blend_file_path = project_path / "blender" / "scene.blend"
script_path = project_path / "src" / "main.py"

In [19]:
def generator(index):
    """See parameters.md for more info"""

    ground_visible = random() > 0.3

    output_path = pth(project_path, "renders")
    assets_path = ppth(project_path, "test_assets")
    texture_path_base = ppth(assets_path, "WoodenPlanks05_MR_2K")

    render_dir_base = "dataset_render_"

    output_path /= f"{render_dir_base}{str(index).zfill(8)}"

    general_conf = {}
    general_conf["resolution"] = (1024, 1024)
    general_conf["compression_ratio"] = 70
    general_conf["render_engine"] = "cycles"
    general_conf["cycles_device"] = "gpu"
    general_conf["cycles_samples"] = 8
    general_conf["cycles_denoise"] = True
    general_conf["output_path"] = str(output_path)

    ground_conf = {}
    ground_conf["visible"] = ground_visible
    ground_conf["offset"] = uniform(-10, 10)
    ground_conf["texture_rotation"] = uniform(0, 360)
    ground_conf["displacement_strength"] = uniform(0.04, 0.2)
    ground_conf["subdivisions"] = 9

    for name, type in zip(
        (
            "WoodenPlanks05_2K_BaseColor.png",
            "WoodenPlanks05_2K_Roughness.png",
            "WoodenPlanks05_2K_Height.png",
        ),
        ("albedo", "roughness", "depth"),
    ):
        ground_conf[f"texture_path_{type}"] = str(texture_path_base / name)

    paper_conf = {}
    paper_conf["text_image_path"] = str(ppth(assets_path, "lorem ipsum.psd"))
    paper_conf["size"] = (21.0, 29.7)
    paper_conf["subdivisions"] = 8
    paper_conf["crumpling_strength"] = uniform(0, 1.5)
    paper_conf["fold_messiness"] = uniform(0.03, 0.4)
    paper_conf["fold_smoothness"] = uniform(0, 1)
    paper_conf["texture_rotation"] = uniform(0, 360)
    paper_conf["offset"] = uniform(-10, 10)

    folds_conf = []
    for _ in range(2):
        fold = {}
        fold["strength"] = uniform(0.1, 0.8) if random() > 0.3 else 0.0
        fold["angle"] = uniform(-15, 15)
        folds_conf.append(fold)

    folds_conf[1]["angle"] += 90

    camera_conf = {}
    camera_conf["focal_length"] = randint(24, 135)
    camera_conf["relative_camera_distance"] = 1.3
    camera_conf["depth_of_field"] = True
    camera_conf["fstop"] = uniform(0.8, 1.8)
    camera_conf["orbit"] = (uniform(0, 25), uniform(0, 360))
    camera_conf["look_at_2d"] = (0, 0)

    hdri_conf = {}
    hdri_conf["hdri_image_path"] = str(assets_path / "canary_wharf_2k.exr")
    hdri_conf["hdri_light_strength"] = uniform(0.02, 0.2)
    hdri_conf["hdri_backdrop_strength"] = 1.0
    hdri_conf["hdri_image_rotation"] = uniform(0, 360)

    lights_conf = []
    for _ in range(2):
        light = {}
        light["visible"] = True

        light["distance"] = uniform(2, 4)
        light["orbit"] = uniform(0, 45), uniform(0, 360)
        light["look_at_2d"] = uniform(-0.4, 0.4), uniform(-0.4, 0.4)

        light["power"] = uniform(500, 900)
        light["shadow_softness_radius"] = uniform(0.1, 0.8)
        light["light_cone_angle"] = uniform(30, 90)

        color = [uniform(0.7, 1) for _ in range(3)]
        color_total = sum(color)
        color = [c / color_total for c in color]
        light["color"] = color
        lights_conf.append(light)

    lights_conf[1]["visible"] = random() > 0.7
    if lights_conf[1]["visible"]:
        # both lights are visible, need to reduce light power
        for light in lights_conf:
            light["power"] /= 2

    conf = {
        "general": general_conf,
        "ground": ground_conf,
        "paper": paper_conf,
        "folds": folds_conf,
        "camera": camera_conf,
        "hdri": hdri_conf,
        "lights": lights_conf,
    }

    return conf

In [24]:
n_samples_to_generate = 2
samples = [generator(i) for i in range(n_samples_to_generate)]
config.write(samples)

In [25]:
!"$blender_path" "$blend_file_path" --background --factory-startup --threads 0 --engine CYCLES --enable-autoexec --python "$script_path" -- --cycles-device "OPTIX"

Blender 3.4.0 (hash a95bf1ac01be built 2022-12-07 02:06:25)
Read blend: f:\GbotHQ\Documents\GitHub\Blender-3D-document-rendering-pipeline\blender\scene.blend
Generating 2 samples...
Failed to enable OPTIX, falling back to CUDA (when rendering from a command line, use [--engine CYCLES] with [--cycles-device OPTIX] and ignore this message)
Failed to enable CUDA, falling back to CPU (when rendering from a command line, use [--engine CYCLES] with [--cycles-device CUDA] and ignore this message)
Fra:1 Mem:34.04M (Peak 50.21M) | Time:00:00.81 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | paper
Fra:1 Mem:43.93M (Peak 50.21M) | Time:00:00.94 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Initializing
Fra:1 Mem:30.79M (Peak 50.21M) | Time:00:00.94 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Waiting for render to start
Fra:1 Mem:30.79M (Peak 50.21M) | Time:00:00.95 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Loading render kernels (may take a few minutes the first time)
